# Risk Rating #

+ How to determine credit rating of borrower?

+ Financial Analysis
    + 3-year financial statements
    + isolate 3/4/5 key factors
    + tailored to company / industry segment where possible
        + for now, just general and real estate holdco models
+ Qualitative Factors: customer relationship, historical payment record
    + up/downgrade based on subjective assessment

#### CODE REQUIRES ####

+ Base Risk Model Template
+ Child Models that inheret
+ Standardized driver calculations
+ Each model needs transform equation between driver values and relative contribution
    + each driver has:
        + min and max values to establish bounded range
        + transformation to 1 - 10 scale; 1 being lowest risk and 10 being highest risk

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd

In [3]:
from htsfi.rating import drivers
from htsfi.rating.models import AllIndustries, Wholesale, Service

In [4]:
rev = 350
cogs = 340
opex = 65
amort = 100
ebitda = rev - cogs - opex + amort
ffo = ebitda
interest = 50
otherinc = 25
nibt = ebitda - amort - interest + otherinc
r_n_d = 50
capleases = 75

debt = 500
capital = 350
total_assets = 1000

In [5]:
allind = AllIndustries(
    amort, capital, capleases, cogs, debt, ebitda, ffo,
    interest, nibt, opex, otherinc, r_n_d, rev,
    total_assets
)

In [6]:
allind.display()

0.0053 51 BB+


In [4]:
xl = pd.ExcelFile('historical.xlsx')

In [5]:
istat = xl.parse('istat', index_col=0)
bs = xl.parse('bs', index_col=0)

In [6]:
split_rows = np.argwhere(bs.isna().all(axis=1).values)

In [7]:
current_assets, lt_assets, curr_l, lt_l, equity = np.split(bs, split_rows.flatten(), axis=0)

In [8]:
capital = equity.dropna().sum()[2020]

debt = (lt_l.dropna().sum() + curr_l.iloc[-3:-1].sum())[2020]
debt = (equity.iloc[1][2020] + debt)

ebitdas = istat.loc['Revenue'] - istat.loc['COGS'] - istat.loc['Opex'] + istat.loc['Otherinc']
ebitda = ebitdas[2020]

interest = istat.loc['interest', 2020]
rev = istat.loc['Revenue', 2020]
rev_1 = istat.loc['Revenue', 2019]
total_assets = current_assets[2020].sum() + lt_assets[2020].sum()

In [9]:
whole = Wholesale(capital, debt, ebitda, interest, rev, rev_1, total_assets)

In [10]:
whole.display()

0.0017 41 BBB+


In [178]:
whole.drivers

[Interest Cover: 28.911253430924063 / 1.3266239707227818,
 EBITDA Margin: 0.05808823529411765 / 7.970852215231642,
 Revenue Growth: -0.005484460694698323 / 6.846039165693783,
 Debt to Capital: 0.11532128947278777 / 2.092517479215884,
 Total Assets: 5.381154627356524 / 5.086860759419617]

In [25]:
multiple = 5
debt_ead = 48

In [27]:
coll_cov = ebitda*5*.5 / debt_ead

In [29]:
coll_cov_ev = [.37, .53, .63, .77, .97, 1.75, 3]
coll_cov_ev = np.stack((coll_cov_ev[:-1], coll_cov_ev[1:])).T

In [30]:
lgd_rating_ev = ['K', 'J', 'I', 'H', 'G', 'F', 'E', 'D']
"""5 year average EBITDA as multiple of outstanding debt / discounted by .3 - .7"""

'5 year average EBITDA as multiple of outstanding debt / discounted by .3 - .7'

In [31]:
coll_cov_ev

array([[0.37, 0.53],
       [0.53, 0.63],
       [0.63, 0.77],
       [0.77, 0.97],
       [0.97, 1.75],
       [1.75, 3.  ]])

In [41]:
if coll_cov < coll_cov_ev[0,0]:
    idx = 0  
else: 
    idx = len(lgd_rating_ev) - 1
    for i in range(1, coll_cov_ev.shape[0]):
        bounds = coll_cov_ev[i-1]
        print (i, bounds)
        if coll_cov >= bounds[0] and coll_cov < bounds[1]:
            idx = i
            break

1 [0.37 0.53]
2 [0.53 0.63]
3 [0.63 0.77]
4 [0.77 0.97]
5 [0.97 1.75]


In [42]:
lgd_rating_ev[idx]

'F'

In [43]:
"Properites: resi: 75% LGD / Commerical/INdustrial 60% with +/- 5/10% on various characteristics"


'SAME GRID AS EV!!!!'

In [51]:
amort = istat.loc['Amort', 2020]

In [54]:
ar = current_assets.loc['A/R', 2020]

In [55]:
serv = Service(
    amort, ar, capital, capleases, cogs, debt, ebitda, interest, nibt,
    opex, otherinc, r_n_d, rev, total_assets
)

In [56]:
serv.display()

0.0034 47 BBB-
